In [1]:
import pandas as pd

# Sample data for dataset d1 (Authors per paper)
df_d1 = pd.read_excel('C:/Users/HP/Desktop/Athar_Author_New.xlsx', header = 0)

# Sample data for dataset d2 (Citations)
df_d2 = pd.read_excel('C:/Users/HP/Desktop/SentiWordNet_Citation.xlsx', header = 0)
print(df_d2)

      Unnamed: 0    Source    target sentiment  \
0              0  A00-1043  A00-2024         o   
1              1  H05-1033  A00-2024         o   
2              2  I05-2009  A00-2024         o   
3              3  I05-2009  A00-2024         o   
4              4  I05-2009  A00-2024         o   
...          ...       ...       ...       ...   
8731        8731  W98-1118  W96-0213         p   
8732        8732  W99-0606  W96-0213         o   
8733        8733  W99-0607  W96-0213         o   
8734        8734  W99-0607  W96-0213         p   
8735        8735  W99-0608  W96-0213         o   

                                               sentence  p_score  
0     We analyzed a set of articles and identified s...    0.000  
1     Table 3: Example compressions Compression AvgL...    0.000  
2     5.3 Related works and discussion Our two-step ...    0.500  
3     (1999) proposed a summarization system based o...    0.500  
4     We found that the deletion of lead parts did n...    0.000

In [2]:
"""reshaping the data to the form in which every instance contains the article id and its corresponding 
author numbers (each instance contaning one author number 1-14) and that author's name """
data2_long = df_d1.melt(id_vars=['Paper'], value_vars=df_d1.columns[1:], var_name='AuthorNumber', value_name='Author')
data2_long

,Paper,AuthorNumber,Author
0,A00-1004,Author1,"Chen, Jiang"
1,A00-1005,Author1,"Bagga, Amit"
2,A00-1007,Author1,"Jonsson, Arne"
3,A00-1011,Author1,"Aone, Chinatsu"
4,A00-1012,Author1,"Stevenson, Mark"
...,...,...,...
42961,W99-0707,Author14,NaN
42962,W99-0903,Author14,NaN
42963,W99-0905,Author14,NaN
42964,W99-0908,Author14,NaN


In [3]:
# Removing NULL values
data2_long.fillna('', inplace=True)
data2 = data2_long[data2_long['Author']!= '']

In [4]:
data2

,Paper,AuthorNumber,Author
0,A00-1004,Author1,"Chen, Jiang"
1,A00-1005,Author1,"Bagga, Amit"
2,A00-1007,Author1,"Jonsson, Arne"
3,A00-1011,Author1,"Aone, Chinatsu"
4,A00-1012,Author1,"Stevenson, Mark"
...,...,...,...
34819,N04-1021,Author12,"Jin, Zhen"
35448,P07-2045,Author12,"Bojar, Ondřej"
37888,N04-1021,Author13,"Radev, Dragomir"
38517,P07-2045,Author13,"Constantin, Alexandra"


In [5]:
##### Checking for Self-Citations, pair of Citing and Cited Articles sharing atleast one author ##########
ndf = []
for i in df_d2.index:
    citing_df = data2[data2['Paper'] == df_d2['Source'][i]]
    cited_df =  data2[data2['Paper'] == df_d2['target'][i]]
    for j in citing_df['Author']:
        for k in cited_df['Author']:
            ndf.append([df_d2['Source'][i], j, df_d2['target'][i], k, df_d2['sentence'][i], df_d2['p_score'][i]])
df = pd.DataFrame(ndf)

In [6]:
new_data = df[df[1]==df[3]]

In [7]:
new_data

,0,1,2,3,4,5
0,A00-1043,"Jing, Hongyan",A00-2024,"Jing, Hongyan",We analyzed a set of articles and identified s...,0.000
65,J05-3002,"McKeown, Kathleen",A00-2024,"McKeown, Kathleen",As previously observed in the literature (Mani...,-0.250
69,J05-3002,"McKeown, Kathleen",A00-2024,"McKeown, Kathleen","In addition to sentence fusion, compression al...",0.000
73,J05-3002,"McKeown, Kathleen",A00-2024,"McKeown, Kathleen",While earlier approaches for text compression ...,0.000
77,J05-3002,"McKeown, Kathleen",A00-2024,"McKeown, Kathleen",While this approach exploits only syntactic an...,0.000
...,...,...,...,...,...,...
52042,A97-1004,"Ratnaparkhi, Adwait",W96-0213,"Ratnaparkhi, Adwait",4 Maxilnum Entropy The model used here for sen...,0.000
52427,P98-2177,"Ratnaparkhi, Adwait",W96-0213,"Ratnaparkhi, Adwait","The tagger from (Ratnaparkhi, 1996) first anno...",0.000
52622,W97-0301,"Ratnaparkhi, Adwait",W96-0213,"Ratnaparkhi, Adwait",The maximum entropy models used here are simil...,0.000
52623,W97-0301,"Ratnaparkhi, Adwait",W96-0213,"Ratnaparkhi, Adwait",The training samples are respectively used to ...,0.125


In [8]:
new_data.columns = ['Citing PaperID', 'Citing Author', 'Cited PaperID', 'Cited Author', 'Sentence', 'P_Score']

In [10]:
# Storing Self-Citations
new_data.to_excel('C:/Users/HP/Desktop/Author_Self_Citations.xlsx', index = False)

In [ ]:
new_data.loc[,'C_Id'] = new_data.loc[,'Citing PaperID'] + new_data.loc[,'Cited PaperID']

In [ ]:
import pandas as pd

# Merging df and new_data on the appropriate columns
merged_data = pd.merge(df, new_data, left_on=[0, 2], right_on=['Citing PaperID', 'Cited PaperID'], how='left', indicator=True)

# Filtering out the rows where the combination of df[0][i] and df[2][i] exists in new_data
# This will remove Self-Citiations
filtered_df = merged_data[merged_data['_merge'] == 'left_only']

# Droping the columns used for merging and the indicator column
filtered_df.drop(['Citing Author', 'Cited Author', 'Sentence', 'P_Score', 'C_Id', 'Citing PaperID', 'Cited PaperID', '_merge'], axis=1, inplace=True)


In [13]:
# This contains edges between Articles for Each Author
filtered_df

,0,1,2,3,4,5
2,H05-1033,"Sporleder, Caroline",A00-2024,"Jing, Hongyan",Table 3: Example compressions Compression AvgL...,0.000
3,H05-1033,"Sporleder, Caroline",A00-2024,"McKeown, Kathleen",Table 3: Example compressions Compression AvgL...,0.000
4,H05-1033,"Lapata, Mirella",A00-2024,"Jing, Hongyan",Table 3: Example compressions Compression AvgL...,0.000
5,H05-1033,"Lapata, Mirella",A00-2024,"McKeown, Kathleen",Table 3: Example compressions Compression AvgL...,0.000
6,I05-2009,"Tanaka, Hideki",A00-2024,"Jing, Hongyan",5.3 Related works and discussion Our two-step ...,0.500
...,...,...,...,...,...,...
87271,W99-0607,"Zhang, Ruiqiang",W96-0213,"Ratnaparkhi, Adwait",Our model exploits the same kind of tag-n-gram...,0.625
87272,W99-0608,"Marquez, Lluis",W96-0213,"Ratnaparkhi, Adwait","In that table, TBL stands for Brill's transfor...",0.000
87273,W99-0608,"Rodriguez, Horacio",W96-0213,"Ratnaparkhi, Adwait","In that table, TBL stands for Brill's transfor...",0.000
87274,W99-0608,"Carmona, Josep",W96-0213,"Ratnaparkhi, Adwait","In that table, TBL stands for Brill's transfor...",0.000


In [14]:
filtered_df.columns = ['Citing Paper', 'Citing Author', 'Cited Paper', 'Cited Author', 'Sentence', 'P_Score']

In [15]:
filtered_df.to_excel('C:/Users/HP/Desktop/Author_EdgeList_Filtered.xlsx', index = False)